In [1]:
parent_folder = '/Users/ravi.tej/Desktop/ML/Recommendations/arcane/'
from hydra import compose, initialize
import os

import xml.etree.ElementTree as ET

tree = ET.parse('../../conf/application.run.xml')
root = tree.getroot()

envs_element = root.find('./configuration/envs')
for variable in envs_element.findall('env'):
    name = variable.get('name')
    value = variable.get('value')
    os.environ[name] = value

import sys
sys.path.append('/Users/ravi.tej/Desktop/ML/Recommendations/arcane/')

from src._utils import load_bertopic_model_from_hf

In [2]:
import requests

In [3]:
r = requests.post('http://Carbine-env.eba-tgfee4hb.ap-south-1.elasticbeanstalk.com/get_recent_published_document_ids', json = {'num_days': 2})

In [ ]:
r.text

In [9]:
requests.post('http://Arcane-env.eba-mrsaixmg.ap-south-1.elasticbeanstalk.com/generate_new_clusters_to_s3')

<Response [504]>

In [2]:
with initialize(config_path="../../conf"):
    # Compose the configuration
    cfg = compose(config_name="ClustersResetService.yaml")

In [3]:
from src.articles.ArticleService import ArticleService

In [17]:
ArticleService.get_Article('65316e121e5cc42b1b13e420')

Article(article_id='65316e121e5cc42b1b13e420', title='Improving GST | Mint', cleaned_text=' Three months into the goods and services tax (GST), the GST Council’s recalibration on Friday offered some immediate relief but left a number of questions unanswered. Both cutting rates on 27 GST items and easing up on exporters are welcome steps. As for lowering compliance costs for small enterprises, it is strange that the burden of filing returns on a monthly basis was recognized only now. It would also be wise to keep in mind the possibility of unintended consequences—seen with other micro, small and medium enterprises regulations that provide perverse incentives to stay small. In the longer run, shuffling items from one GST slab to another will not be enough. Finance minister Arun Jaitley has spoken of eventually reducing the number of slabs. This is essential. So are other structural changes like getting rid of the anti-profiteering provisions, which are a bad idea through and through. And

In [3]:
from src.clustering.ClustersResetService import ClustersResetService

- how does Qlora work
- how is QA lora different
- why do adapter network add high latency overhead
- what is the meaning of 'intrinsic' dimension
- why not use the lora technique for pre-training of the model?
- how is that f32 takes 4 bits, f16 2, int8 - 1, int4 - 1/2?
- how do the weights of an LLM look like? whats the distribution?

In [5]:
# from sql.PostgresDatabaseOperation import PostgresDatabaseOperation

# def create_table_backup(table_name):
#     backup_table_name = f'{table_name}_backup'
#     with PostgresDatabaseOperation() as cursor:
#         create_backup_table_sql = f"CREATE TABLE {backup_table_name} AS TABLE {table_name};"
#         cursor.execute(create_backup_table_sql)

# for table in ['article_to_cluster_mapping', 'cluster_hierarchy', 'storyline_to_cluster_mapping', 'topic_cluster_preference_mapping']:
#     create_table_backup(table)

In [4]:
clusters = ClustersResetService(cfg=cfg, random_state=86)

fetched all embeddings
proceeding to fetch article details


article details fetched
bertopic model instantiated


2023-11-08 16:14:53,549 - BERTopic - Reduced dimensionality
2023-11-08 16:15:31,059 - BERTopic - Clustered reduced embeddings


model fit to documents


100%|██████████| 169/169 [00:05<00:00, 30.04it/s]


ctfidf_config.json:   0%|          | 0.00/115M [00:00<?, ?B/s]

Upload 3 LFS files:   0%|          | 0/3 [00:00<?, ?it/s]

ctfidf.safetensors:   0%|          | 0.00/101M [00:00<?, ?B/s]

topic_embeddings.safetensors:   0%|          | 0.00/701k [00:00<?, ?B/s]

docs assigned to balanced clusters


In [7]:
from src._utils import save_bertopic_model_to_s3

In [9]:
save_bertopic_model_to_s3(model=clusters.bertopic_model, run_id=clusters.run_id)

In [5]:
clusters.run_id

'RosemaryJackalReindeer'

In [11]:
df = clusters.cluster_hierarchy.hierarchy_df

In [6]:
len(clusters.cluster_hierarchy.balanced_clusters)

28

In [18]:
from sql.PostgresDatabaseOperation import PostgresDatabaseOperation

In [21]:
start_time = time.time()
save_article_story_cluster_mapping(clusters.article_story_cluster_map, clusters.run_id)
print(f'took {int(time.time() - start_time)/60} minutes to finish')

took 27.75 minutes to finish


In [20]:
def save_article_story_cluster_mapping(article_story_cluster_mapping: dict, clustering_run_id: str):
    def chunked_data(data, size):
        """Yield successive chunk_size-sized chunks from data."""
        for i in range(0, len(data), size):
            yield data[i:i + size]

    with PostgresDatabaseOperation() as cursor:
        # Create a temporary table
        cursor.execute("CREATE TABLE temp_article_to_cluster_mapping (LIKE article_to_cluster_mapping INCLUDING ALL)")

        # Prepare data for bulk insert
        bulk_data = []
        for article_id, article_dict in article_story_cluster_mapping.items():
            bulk_data.append((
                article_id,
                int(article_dict['storyline_id']),
                int(article_dict['cluster_id']),
                clustering_run_id,
                float(article_dict['storyline_prob']),
                int(article_dict['story_cluster_id']),
                int(article_dict['max_agg_cluster_id']),
                float(article_dict['agg_cluster_prob'])
            ))

        # Bulk insert in chunks
        insert_sql = f"""INSERT INTO temp_article_to_cluster_mapping 
                         (article_id, storyline_id, cluster_id, clustering_run_id, storyline_prob, story_cluster_id, max_agg_cluster_id, agg_cluster_prob)
                         VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
                      """
        chunk_size = 10000
        for chunk in chunked_data(bulk_data, chunk_size):
            cursor.executemany(insert_sql, chunk)
        
        cursor.execute("BEGIN")
        # Rename tables
        cursor.execute("ALTER TABLE article_to_cluster_mapping RENAME TO article_to_cluster_mapping_old")
        cursor.execute("COMMIT")
        # cursor.execute("CREATE TABLE article_to_cluster_mapping AS TABLE temp_article_to_cluster_mapping")
        cursor.execute("ALTER TABLE temp_article_to_cluster_mapping RENAME TO article_to_cluster_mapping")
        cursor.execute("DROP TABLE article_to_cluster_mapping_old CASCADE")

In [22]:
from sql.clustering.ClusteringSQL import ClusteringSQL

In [25]:
from copy import deepcopy

In [26]:
renamed_hierarchy_df = deepcopy(clusters.cluster_hierarchy.hierarchy_df)

In [27]:
renamed_hierarchy_df = renamed_hierarchy_df.rename(columns = {'cluster_id': 'parent_id', 'cluster_name': 'parent_name', 'storylines': 'child_storyline_list',
                                      'left_child_id': 'child_left_id', 'left_child_name': 'child_left_name', 
                                       'right_child_id': 'child_right_id','right_child_name': 'child_right_name'})

In [28]:
ClusteringSQL.insert_cluster_hierarchy(cluster_hierarchy_df=renamed_hierarchy_df, clustering_run_id=clusters.run_id)

In [29]:
ClusteringSQL.insert_storyline_to_cluster_mapping(story_to_cluster_mapping=clusters.cluster_hierarchy.story_cluster_map,
                                                 clustering_run_id=clusters.run_id)

In [32]:
insert_story_embeddings(story_embeddings=clusters.bertopic_model.topic_embeddings_, clustering_run_id=clusters.run_id)

In [31]:
def insert_story_embeddings(story_embeddings, clustering_run_id):
    with PostgresDatabaseOperation() as cursor:
        truncate_sql = f"""
                        TRUNCATE TABLE story_embeddings RESTART IDENTITY
                        """
        cursor.execute(truncate_sql)

        insert_sql = f"""INSERT INTO story_embeddings (story_id, story_embedding, clustering_run_id)
                        VALUES (%s, %s, %s)
                        """
        # TODO: - can't ideally assume that -1 exists. should come from clustering run - refactor
        for i in range(-1, len(story_embeddings) - 1):
            story_embedding = list(story_embeddings[i + 1])
            cursor.execute(insert_sql, (i, story_embedding, clustering_run_id))

In [33]:
from src._utils import save_bertopic_model_to_hf, save_bertopic_model_to_s3

In [35]:
save_bertopic_model_to_s3(model = clusters.bertopic_model, run_id=clusters.run_id)

In [59]:
import joblib

In [61]:
!pip install --upgrade joblib

  Obtaining dependency information for joblib from https://files.pythonhosted.org/packages/10/40/d551139c85db202f1f384ba8bcf96aca2f329440a844f924c8a0040b6d02/joblib-1.3.2-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.2/302.2 kB 337.5 kB/s eta 0:00:00a 0:00:01
  Attempting uninstall: joblib
    Found existing installation: joblib 1.2.0
    Uninstalling joblib-1.2.0:
      Successfully uninstalled joblib-1.2.0


In [65]:
import tempfile
import joblib
import boto3

In [66]:
import os

In [68]:
clusters.bertopic_model

In [69]:
from copy import deepcopy

In [71]:
from bertopic import BERTopic

In [ ]:
# WRITE
with tempfile.TemporaryFile() as fp:
    joblib.dump(model, fp)
    fp.seek(0)
    s3_client.put_object(Body=fp.read(), Bucket=bucket_name, Key=key)

In [73]:
s3_bucket = 'insight-ml-models'
s3_bertopic_folder = 'bertopic'

In [77]:
def save_bertopic_model_to_s3_joblib(model: BERTopic, run_id: str):
    filename = 'BERTopic_' + run_id
    model = deepcopy(model)
    model.embedding_model = None
    s3_resource = boto3.resource('s3', aws_access_key_id=os.environ['AWS_ACCESS_KEY'], aws_secret_access_key=os.environ['AWS_SECRET_KEY'])
    # pickle_byte_obj = pickle.dumps(model)
    with tempfile.TemporaryFile() as fp:
        joblib.dump(model, fp)
        fp.seek(0)
        s3_resource.Object(s3_bucket, f'{s3_bertopic_folder}/{filename}').put(Body=fp.read())

In [78]:
save_bertopic_model_to_s3_joblib(model=clusters.bertopic_model, run_id=clusters.run_id)

In [37]:
save_bertopic_model_to_hf(model = clusters.bertopic_model, run_id=clusters.run_id, is_dev=True)

topic_embeddings.safetensors:   0%|          | 0.00/717k [00:00<?, ?B/s]

ctfidf.safetensors:   0%|          | 0.00/101M [00:00<?, ?B/s]

ctfidf_config.json:   0%|          | 0.00/115M [00:00<?, ?B/s]

Upload 3 LFS files:   0%|          | 0/3 [00:00<?, ?it/s]

In [84]:
s3_resource = boto3.client('s3', aws_access_key_id=os.environ['AWS_ACCESS_KEY'], aws_secret_access_key=os.environ['AWS_SECRET_KEY'])

In [85]:
s3_resource.download_fileobj?

Signature:
s3_resource.download_fileobj(
    Bucket,
    Key,
    Fileobj,
    ExtraArgs=None,
    Callback=None,
    Config=None,
)
Docstring:
Download an object from S3 to a file-like object.

The file-like object must be in binary mode.

This is a managed transfer which will perform a multipart download in
multiple threads if necessary.

Usage::

    import boto3
    s3 = boto3.client('s3')

    with open('filename', 'wb') as data:
        s3.download_fileobj('mybucket', 'mykey', data)

:type Bucket: str
:param Bucket: The name of the bucket to download from.

:type Key: str
:param Key: The name of the key to download from.

:type Fileobj: a file-like object
:param Fileobj: A file-like object to download into. At a minimum, it must
    implement the `write` method and must accept bytes.

:type ExtraArgs: dict
:param ExtraArgs: Extra arguments that may be passed to the
    client operation. For allowed download arguments see
    boto3.s3.transfer.S3Transfer.ALLOWED_DOWNLOAD_ARGS.

:t

In [106]:
from io import BytesIO


In [109]:
def load_bertopic_model_from_s3(run_id: str, hf_embedding_model_name: str) -> BERTopic:
    filename = 'BERTopic_' + run_id
    s3_resource = boto3.resource('s3', aws_access_key_id=os.environ['AWS_ACCESS_KEY'], aws_secret_access_key=os.environ['AWS_SECRET_KEY'])
    # s3_object = s3_resource.Object(s3_bucket, f'{s3_bertopic_folder}/{filename}')
    # s3_client = boto3.client('s3', aws_access_key_id=os.environ['AWS_ACCESS_KEY'], aws_secret_access_key=os.environ['AWS_SECRET_KEY'])
    # response = s3_client.get_object(Bucket=s3_bucket, Key=f'{s3_bertopic_folder}/{filename}')
    with BytesIO() as data:
        s3_resource.Bucket(s3_bucket).download_fileobj(f'{s3_bertopic_folder}/{filename}', data)
        print('preparing to seek')
        data.seek(0)    # move back to the beginning after writing
        print('seeked')
        model = joblib.load(data)
        print('loading now')
    # Load the model from the response Body using joblib
    # model = joblib.load(response['Body'].read())
    assert isinstance(model, BERTopic), f"Filed to load model from S3. model {filename} is not of type BERTopic. "
    model.embedding_model = SentenceTransformerBackend(embedding_model=hf_embedding_model_name)
    return model

In [104]:
# with open('/tmp/' + fileName, 'wb') as data:      
#     s3.Bucket("bucket-name").download_fileobj(fileName, data)  
# with open('/tmp/' + fileName, 'rb') as data:      
#     model = joblib.load(data)  return model

In [110]:
some_model = load_bertopic_model_from_s3(run_id=clusters.run_id, hf_embedding_model_name='BAAI/bge-large-en-v1.5')

preparing to seek
seeked
loading now


In [112]:
some_model.embedding_model.embed_words('something something big words')

array([ 0.03268695, -0.01644479,  0.02162185, ...,  0.01125305,
       -0.0424784 ,  0.0104564 ], dtype=float32)

In [ ]:
def 
# READ
with tempfile.TemporaryFile() as fp:
    s3_client.download_fileobj(Fileobj=fp, Bucket=bucket_name, Key=key)
    fp.seek(0)
    model = joblib.load(fp)

In [42]:
from src.topics.TopicClusterMapping import TopicClusterMapping

In [44]:
from sql.PostgresDatabaseOperation import PostgresDatabaseOperation

In [47]:
from bertopic.backend._sentencetransformers import SentenceTransformerBackend

In [48]:
emb_model = SentenceTransformerBackend('BAAI/bge-large-en-v1.5')

In [49]:
clusters.bertopic_model.embedding_model = emb_model

In [52]:
_trigger_topic_preferences_recalculation(run_id=clusters.run_id, bertopic_model=clusters.bertopic_model)

In [40]:
def _trigger_topic_preferences_recalculation(run_id, bertopic_model):
    with initialize(config_path="../../conf"):
        # Compose the configuration
        topic_cfg = compose(config_name="TopicClusterMapping.yaml")
    topic_cluster_mapping = TopicClusterMapping(clustering_run_id=run_id, bertopic_model=bertopic_model, cfg=topic_cfg)
    topic_cluster_mapping.recompute_preferences_for_all_topics()

In [57]:
def add_new_source_details(source_id, source_name,publication_date_decay):
    with PostgresDatabaseOperation() as cursor:
        insert_sql = """
        INSERT INTO source_id_characteristics (source_id, publication_date_decay, source_name)
        VALUES (%s, %s, %s)
        ON CONFLICT (source_id)
        DO UPDATE SET
            publication_date_decay = EXCLUDED.publication_date_decay,
            source_name = EXCLUDED.source_name
        """
        cursor.execute(insert_sql, (source_id, publication_date_decay, source_name))

In [58]:
add_new_source_details(source_id='6512cdcad01a9c8e86263e05', source_name='economic-times', publication_date_decay=0.9)

In [8]:
clusters.cluster_hierarchy.balanced_clusters

[6, 50, 73, 74, 71, 67, 68, 72, 79]

In [38]:
clusters.documents

array(['Messi Meets America Documentary Series: Heres release schedule streaming platform and everything you m: Lionel Messi the famous footballer played his first match for Inter Miami FC in Major League Soccer in July. It was his North American debut.A documentary series titled Messi Meets America chronicles Messis transition from international football fame to playing for a less-known team in a less popular league. Messis move to Miami had a significant impact on soccer viewership in North America even prompting Adidas to release special T-shirts in his honor.Messi Meets America Documentary Series: Release Schedule Streaming PlatformThis documentary consists of six episodes. The first three episodes are currently available for streaming on Apple TV. The fourth episode will be available for streaming on November 1. Release dates for episodes five and six are yet to be announced.Messi Meets America Documentary Series: Production TeamThe series includes interviews with David Beckham wh

In [7]:
from src.clustering.ClusterHierarchyService import ClusterHierarchyService

In [6]:
clusters.bertopic_model

In [13]:
import numpy as np

In [17]:
np.argmax(clusters.bertopic_model.probabilities_[0])

865

In [18]:
np.max(clusters.bertopic_model.probabilities_[1])

0.07908182017400811

In [9]:
chs = ClusterHierarchyService(documents=clusters.documents, bertopic_model=clusters.bertopic_model,cfg=clusters.cfg)

100%|██████████| 1387/1387 [00:46<00:00, 30.11it/s]


In [24]:
from src.clustering.ClusterHierarchyService import ClusterHierarchyService

SyntaxError: invalid syntax (2783995047.py, line 1)

In [28]:
chs.hierarchy_df

,cluster_id,cluster_name,storylines,left_child_id,left_child_name,right_child_id,right_child_name,distance
0,2774,change_net change_stock price_percent change_p...,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",2772,cent_real_estate_real estate_quarter,2773,change_net change_stock price_percent change_s...,5.913270
1,2773,change_net change_stock price_percent change_s...,"[31, 67, 77, 86, 106, 126, 131, 135, 137, 139,...",2748,divis_divis lab_lab_live blog_blog,2766,change_net change_stock price_percent change_s...,4.711992
2,2772,cent_real_estate_real estate_quarter,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",2771,real_estate_real estate_quarter_commercial,2767,nifty_gold_index_dollar_inflation,3.571896
3,2771,real_estate_real estate_quarter_commercial,"[0, 2, 6, 7, 8, 10, 11, 13, 14, 15, 17, 18, 19...",2769,real_estate_real estate_residential_property,2770,ipo_quarter_yoy_dividend_revenue,2.855762
4,2770,ipo_quarter_yoy_dividend_revenue,"[2, 10, 14, 17, 19, 23, 24, 26, 27, 30, 38, 39...",2668,ipo_issue_gmp_subscribed_allotment,2759,quarter_yoy_dividend_revenue_net profit,2.719312
...,...,...,...,...,...,...,...,...
2771,1384,1383,[],None,,None,,NaN
2772,1385,1384,[],None,,None,,NaN
2773,1386,1385,[],None,,None,,NaN
2774,1387,1386,[],None,,None,,NaN


In [31]:
row = chs.hierarchy_df[chs.hierarchy_df.cluster_id == 2774].squeeze()

In [32]:
row

cluster_id                                                       2774
cluster_name        change_net change_stock price_percent change_p...
storylines          [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...
left_child_id                                                    2772
left_child_name                  cent_real_estate_real estate_quarter
right_child_id                                                   2773
right_child_name    change_net change_stock price_percent change_s...
distance                                                      5.91327
Name: 0, dtype: object

In [33]:
left_child_id = row.left_child_id

In [34]:
rigth_child_id = row.right_child_id

In [36]:
levels = {2774: 0}

In [37]:
levels.update({})

In [26]:
chs._populate_levels(parent_cluster_id=2774,parent_level=0)

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [19]:
chs.setup()

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [10]:
chs.hierarchy_df

,Parent_ID,Parent_Name,Topics,Child_Left_ID,Child_Left_Name,Child_Right_ID,Child_Right_Name,Distance
1386,2774,change_net change_stock price_percent change_p...,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",2772,cent_real_estate_real estate_quarter,2773,change_net change_stock price_percent change_s...,5.913270
1385,2773,change_net change_stock price_percent change_s...,"[31, 67, 77, 86, 106, 126, 131, 135, 137, 139,...",2748,divis_divis lab_lab_live blog_blog,2766,change_net change_stock price_percent change_s...,4.711992
1384,2772,cent_real_estate_real estate_quarter,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",2771,real_estate_real estate_quarter_commercial,2767,nifty_gold_index_dollar_inflation,3.571896
1383,2771,real_estate_real estate_quarter_commercial,"[0, 2, 6, 7, 8, 10, 11, 13, 14, 15, 17, 18, 19...",2769,real_estate_real estate_residential_property,2770,ipo_quarter_yoy_dividend_revenue,2.855762
1382,2770,ipo_quarter_yoy_dividend_revenue,"[2, 10, 14, 17, 19, 23, 24, 26, 27, 30, 38, 39...",2668,ipo_issue_gmp_subscribed_allotment,2759,quarter_yoy_dividend_revenue_net profit,2.719312
...,...,...,...,...,...,...,...,...
4,1392,bharti_bharti airtel_airtel_bharti airtels_air...,"[176, 1197]",1197,bharti_bharti airtel_airtel_data bharti_bharti...,176,bharti_bharti airtel_airtel_bharti airtels_air...,0.105899
3,1391,ultra cem_cem_ultra_cem stock_price ultra,"[289, 989]",989,ultra cem_cem_ultra_cem stock_price ultra,289,ultra cem_cem_ultra_cem stock_price ultra,0.103334
2,1390,divis_divis lab_lab_lab stock_price divis,"[527, 588]",527,divis_divis lab_lab_lab stock_price divis,588,divis_divis lab_lab_price divis_lab stock,0.091630
1,1389,04 jul_blog 04_jul 2023_jul_mint disclaimer,"[31, 852]",852,blog 04_04 jul_jul 2023_jul_mint disclaimer,31,04 jul_blog 04_jul 2023_jul_staff download,0.057531


In [7]:
clusters

NameError: name 'clusters' is not defined

In [ ]:
# from src.articles.ArticleService import ArticleService

# ArticleService.get_Article('65316eaf1e5cc42b1b13e623')